# PDAS CA2 


In [ ]:
# Your existing code
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

def load_csv_files(file_names):
    dataframes = []
    for file_names in file_names:
        df = pd.read_csv(file_names)
        dataframes.append(df)
    return dataframes

file_names = ['./greenhousepollution.csv', './solidwastemanagement.csv', './LicensedFoodEstablishmentsbyCategoryAnnual.csv']
dataframes = load_csv_files(file_names)


df1 = dataframes[0]
df2 = dataframes[1]
df3 = dataframes[2]

# nature of datasets
print("Summary of Greenhouse gas pollution")
print(df1.info())
print(df1.head())

print("Summary of Solid Waste Management")
print(df2.info())
print(df2.head())

print("Summary of Total Licencensed Food Establishment")
print(df3.info())
print(df3.head())

From the summary of Greenhouse gas pollution, we can see that the DataFrame has 22 entries (rows) and 16 columns. The columns represent different types of greenhouse gases and their emissions measured in metric tons of CO2 equivalent. The ‘Year’ column is of integer type (int64), while the rest of the columns are of float type (float64).
we can also see some values in the DataFrame that have missing values (NaN), as indicated by the count of non-null entries being less than 22 for some columns. Therefore, we need to create a function to handle missing values.

From the summary of Solid Waste Management, we can see that the DataFrame provides information about each column, such as the count of non-null entries, the number of unique entries, the most frequent entry (top), and the frequency of the most frequent entry (freq).
From the summary, we can see that the DataFrame has 11 entries (rows) and 61 columns. The columns represent different types of waste and their quantities generated and recycled over the years. The ‘Year’ column is of integer type (int64), while the rest of the columns are of float type (float64).
we can also see some values in the DataFrame that have missing values (NaN), as indicated by the count of non-null entries being less than 11 for some columns. Therefore, we need to create a function to handle missing values.

From the summary of Total Licensed Food Establishment, We can see that he DataFrame has 25 entries (rows) and 3 columns: ‘year’, ‘level_1’, and ‘Total Licensed Food Establishments’.
The data types for these columns are int64, object, and float64 respectively.
The ‘year’ column contains integer values representing the year.
The ‘level_1’ column seems to contain the string ‘Total Licensed Food Establishments’ for all rows.
The ‘Total Licensed Food Establishments’ column contains float values, which likely represent the number of licensed food establishments for each year. There are 19 non-null entries in this column, suggesting that there may be some missing data.

In [ ]:
# Function to handle missing values
def handle_missing_values(dataframes):
    for i, df in enumerate(dataframes):
        # Check for null values
        print(f"\nBefore imputation for dataframe {i+1}: ")
        print(df.isnull().sum())
        

        # Fill missing values using the specified method
        df_filled = df.copy()
        for column in df.columns:
            if df[column].dtype == 'object':  # Categorical data
                df_filled[column].fillna(df[column].mode()[0], inplace=True)
            else:  # Numeric data
                df_filled[column].fillna(df[column].mean(), inplace=True)

        # Check for missing values after imputation to show whether the datasets has been imputed
        print(f"\nAfter imputation for dataframe {i+1}: ")
        print(df_filled.isnull().sum())

        # Replace the original dataframe in the list with the filled dataframe
        dataframes[i] = df_filled

    # Return the list of filled dataframes
    return dataframes

dataframes = handle_missing_values(dataframes)




### Data Cleaning
- Detecting missing values
  imputed missing values 
- Detect outliers
  - Outliers are detected by using the 1.5*IQR (interquartile range) rule
  - Due to the large number of outliers, the outliers are not removed as it will affect the accuracy of the model
- Merge datasets
- Print head of the dataframe (sample data)
- Export dataframe to csv

In [ ]:
def calculate_and_remove_outliers(dataframes):
    for i, df in enumerate(dataframes):
        # Select numeric columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

        for col in numeric_cols:
            # Calculate number of outliers
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))].shape[0]
            print(f'Number of outliers in {col} in dataframe {i+1}: {outliers}\n')

            # Remove outliers
            df = df[~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))]
            
        # Replace the original dataframe in the list with the outlier-removed dataframe
        dataframes[i] = df

    # Return the list of outlier-removed dataframes
    return dataframes

# Call the function
dataframes = calculate_and_remove_outliers(dataframes)



### Outlier Removal
Since the Number of Outliers in each column is generally small, ive decided to just remove the outlier values.

In [ ]:
# Extract the imputed and outlier-removed dataframes
df1_imputed = dataframes[0]
df2_imputed = dataframes[1]
df3_imputed = dataframes[2]

# Concatenate dataframes horizontally (add columns)
merged_df = pd.concat([df1_imputed.set_index('Year'), df2_imputed.set_index('Year'), df3_imputed.set_index('Year')], axis=1)

# Define independent variable
X = merged_df['Total Licensed Food Establishments']


# Add a constant to the independent value
X1 = sm.add_constant(X)

# Replace inf with nan and drop rows with nan values
X1.replace([np.inf, -np.inf], np.nan, inplace=True)  
X1.dropna(inplace=True)

# Align indices of X1 with merged_df
aligned_index = X1.index.intersection(merged_df.index)
X1 = X1.loc[aligned_index]
merged_df = merged_df.loc[aligned_index]

# Define dependent variables
y1 = merged_df['  Carbon Dioxide (CO2) (Mt CO2-Equivalent)']
y2 = merged_df['Total Greenhouse Gas Emissions (Mt CO2-Equivalent)']

# Impute missing values in the dependent variables
y1.fillna(y1.mean(), inplace=True)
y2.fillna(y2.mean(), inplace=True)

# Check for constant values
if y1.nunique() == 1 or y2.nunique() == 1:
    print("Dependent variables have constant values. Please select different variables.")
else:
    # Apply linear regression models
    model1 = sm.OLS(y1, X1)
    results1 = model1.fit()
    print(results1.summary())

    model2 = sm.OLS(y2, X1)
    results2 = model2.fit()
    print(results2.summary())


For Total Licensed Food Establishments being the indepedent variable and Total Greenhouse Gas Emissions (Mt CO2-Equivalent) being the dependent the model summary results are that the R-squared value is 0.068, which means that only about 6.8% of the variance in the dependent variable (‘Total Greenhouse Gas Emissions’) can be explained by the independent variable (‘Total Licensed Food Establishments’). This is quite low, suggesting that the model does not explain much of the variability in the data.
The adjusted R-squared value is 0.022, which is even lower. The adjusted R-squared takes into account the number of predictors in the model (in this case, just one), and can sometimes be a better indicator of the quality of a model.
The F-statistic is 1.471 and the Prob (F-statistic) is 0.239. The F-statistic is a measure of how much better the model fits the data than a model with no independent variables. The Prob (F-statistic) is the probability of getting an F-statistic as extreme as the one calculated if the null hypothesis is true (i.e., if the model with no independent variables fits the data as well as your model). A Prob (F-statistic) less than 0.05 is often considered statistically significant, but in this case, it’s 0.239, which is not less than 0.05.
Based on these statistics, it seems that the model might not be a good fit for the data

In [ ]:
# Define a new DataFrame with the variables of interest
df = pd.DataFrame({
    'Total Generated Waste': df2_imputed['Total Generated (Tonnes)'],
    'Total Food Establishments': df3_imputed['Total Licensed Food Establishments'],
    'Total Greenhouse Gas Emissions': df1_imputed['Total Greenhouse Gas Emissions (Mt CO2-Equivalent)']
})


# Plot QQ plots for each column
for column in df.columns:
    plt.figure(figsize=(8, 5))
    sm.qqplot(df[column], line='s')
    plt.title(f'QQ Plot for {column}')
    plt.grid(True)
    plt.show()
    
# Method 1: Scatter plot matrix
sns.pairplot(df)
plt.title('Pairplot of Total Generated Waste, Total Food Establishments, and Total Greenhouse Gas Emissions')
plt.show()

# Method 2: Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

# Plotting scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['Total Food Establishments'], df['Total Greenhouse Gas Emissions'], alpha=0.5)
plt.title('Scatter Plot of Total Food Establishments vs Total Greenhouse Gas Emissions')
plt.xlabel('Total Food Establishments')
plt.ylabel('Total Greenhouse Gas Emissions (Mt CO2-Equivalent)')
plt.grid(True)
plt.show()

# Plotting jointplot
plt.figure(figsize=(10, 6))
sns.jointplot(data=df, x='Total Food Establishments', y='Total Greenhouse Gas Emissions', kind='reg', color='purple')
plt.xlabel('Total Food Establishments')
plt.ylabel('Total Greenhouse Gas Emissions (Mt CO2-Equivalent)')
plt.title('Jointplot of Total Food Establishments vs Total Greenhouse Gas Emissions')
plt.show()

# Create a linear regression plot
plt.figure(figsize=(10, 6))
sns.regplot(data=df, x='Total Food Establishments', y='Total Greenhouse Gas Emissions', scatter_kws={'s': 50}, line_kws={'color': 'red'})
plt.xlabel('Total Food Establishments')
plt.ylabel('Total Greenhouse Gas Emissions (Mt CO2-Equivalent)')
plt.title('Linear Regression: Total Food Establishments vs Total Greenhouse Gas Emissions')
plt.show()

# Method 5: Jointplot
sns.jointplot(x='Total Generated Waste', y='Total Food Establishments', data=df, kind='reg')
plt.xlabel('Total Generated Waste')
plt.ylabel('Total Food Establishments')
plt.title('Jointplot: Total Generated Waste vs Total Food Establishments')
plt.show()
